In [ ]:
from bartiq import Routine, sympy_backend, evaluate
from bartiq.compilation import compile_routine, CompilationFlags
from pathlib import Path
import yaml

try:
    root_dir = next(p for p in (Path.cwd().resolve(), *Path.cwd().resolve().parents) if p.parts[-1] == "bartiq")
    with open(root_dir / "tests/compilation/data/df_qref.yaml") as f:
        rout = Routine.from_qref(yaml.safe_load(f), sympy_backend)
except StopIteration as exc:
    raise ValueError(
        "Can't find the `bartiq` root directory! Make sure this notebook is somewhere in the `bartiq` repo."
    ) from exc

compiled_routine = compile_routine(rout, compilation_flags=CompilationFlags.EXPAND_RESOURCES).routine

assignments = {
    "N_spatial": 35,
    "M_r": int(7590 / 298),
    "R": 298,
    "b_as": 10,
    "b_mas": 10,
    "b_givens": 20,
}
df = evaluate(compiled_routine, assignments=assignments).routine

In [ ]:
from bartiq import CompiledRoutine


def child_contributions_to_resource(routine: CompiledRoutine, resource: str = "t_gates"):
    """For a given routine, check the resource cost of its children."""
    return {
        child_name: resource_value
        for child_name, child_routine in routine.children.items()
        if (resource_value := child_routine.resource_values.get(resource, 0))
    }


child_contributions_to_resource(
    df.children["LCU_compute_0"]
    .children["DoubleFactorizationSelect_compute_0"]
    .children["OneElectronTermBlockEncoding_compute_0"]
    .children["MultiplexedAliasSamplingDF_compute_0"]
    .children["MultiplexedRealUSPDF_compute_0"]
    .children["Repeated_had_0_repetition"]
    .children["Repeated_had_0"]
)

{'had_0': 2}

In [22]:
df.find_descendants("Repeated_had_0")

[['LCU_compute_0',
  'DoubleFactorizationSelect_compute_0',
  'OneElectronTermBlockEncoding_compute_0',
  'MultiplexedAliasSamplingDF_compute_0',
  'MultiplexedRealUSPDF_compute_0',
  'Repeated_had_0_repetition',
  'Repeated_had_0'],
 ['LCU_compute_0',
  'DoubleFactorizationSelect_compute_0',
  'OneElectronTermBlockEncoding_compute_0',
  'MultiplexedAliasSamplingDF_uncompute_0',
  'MultiplexedRealUSPDF_uncompute_0',
  'Repeated_had_0_repetition',
  'Repeated_had_0'],
 ['LCU_compute_0',
  'DoubleFactorizationSelect_compute_0',
  'OneElectronTermBlockEncoding_compute_1',
  'MultiplexedAliasSamplingDF_compute_0',
  'MultiplexedRealUSPDF_compute_0',
  'Repeated_had_0_repetition',
  'Repeated_had_0'],
 ['LCU_compute_0',
  'DoubleFactorizationSelect_compute_0',
  'OneElectronTermBlockEncoding_compute_1',
  'MultiplexedAliasSamplingDF_uncompute_0',
  'MultiplexedRealUSPDF_uncompute_0',
  'Repeated_had_0_repetition',
  'Repeated_had_0']]